# THIS FOLLOWING JSON WORKS FOR GENERATING AN EXCEL FILE THAT WORKS


In [2]:
import pandas as pd
import json

# Load the TLD data with multi-level headers
tld_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\filtered_Tld_B2B_modified (1).xlsx", header=[0, 1])
tld_data.columns = [' '.join(col).strip() for col in tld_data.columns.values]

# Load the CVL file without dropping any rows
cvl_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\CVL_B2B Benefit and Business Key.xlsx")

# Create mappings with NaN values handled gracefully
business_need_mapping = cvl_data.set_index('BusinessNeed_Key')['Business Need'].to_dict()
benefit_id_mapping = cvl_data.set_index('Benefit in your country')['Benefits_Key'].to_dict()

# Initialize list to store data for Excel output
excel_data = []

# Process each row in the TLD data
for _, row in tld_data.iterrows():
    product_global_id = row.get("Unnamed: 1_level_0 ProductGlobalID", "").strip()
    product_entity_sys_id = str(row.get("Unnamed: 0_level_0 sys_id", "")).strip()
    
    if not product_global_id or not product_entity_sys_id:
        print(f"Skipping row due to missing ProductGlobalID or sys_id: {row}")
        continue

    # Retrieve existing JSON from "Segmentation json" if present, otherwise create a new one
    existing_json_str = row.get("Unnamed: 3_level_0 Segmentation json", "{}").strip()
    try:
        segmentation_json = json.loads(existing_json_str) if existing_json_str else {}
    except json.JSONDecodeError:
        print(f"Warning: Malformed JSON for ProductGlobalId {product_global_id}. Attempting to repair.")
        # Attempt minimal repair
        if not existing_json_str.startswith("{"):
            existing_json_str = "{" + existing_json_str
        if not existing_json_str.endswith("}"):
            existing_json_str += "}"
        try:
            segmentation_json = json.loads(existing_json_str)
        except json.JSONDecodeError:
            print(f"Failed to repair JSON for ProductGlobalId {product_global_id}. Using default structure.")
            segmentation_json = {
                "ProductGlobalId": product_global_id,
                "ProductEntitySysId": product_entity_sys_id,
                "MarketModels": []
            }

    # Define the French market model with the data from TLD and CVL files
    fr_market_model = {
        "LocaleId": "fr-FR",
        "LocaleName": "French (France)",
        "BusinessNeeds": []
    }

    # Process each BusinessNeed
    for business_need_id, business_need_name in business_need_mapping.items():
        if pd.isna(business_need_id) or pd.isna(business_need_name):
            continue  # Skip if either key or name is missing

        business_need_entry = {
            "BusinessNeedId": str(int(business_need_id)),
            "Benefit": {
                "BenefitId": "",
                "DisplayOrder": "",
                "ReasonToBelieve": "",
                "TechnicalProof": "",
                "LegalNotice": "",
                "ImageUrl": ""
            }
        }

        # Retrieve the BenefitId and other attributes based on column names with suffixes
        benefit_name = row.get(f"{business_need_name} Benefit in your country", "")
        if benefit_name in benefit_id_mapping:
            benefit_id = benefit_id_mapping[benefit_name]
            business_need_entry["Benefit"]["BenefitId"] = benefit_id

        # Define the column names with suffixes to get specific attributes
        display_order_col = f"{business_need_name} Sort order in your country (from 1 to 9)"
        reason_col = f"{business_need_name} Reason to believe in your language"
        proof_col = f"{business_need_name} Technical proof in your language"
        legal_notice_col = f"{business_need_name} Legal notice in your language"
        
        # Retrieve each attribute for the business need
        business_need_entry["Benefit"]["DisplayOrder"] = row.get(display_order_col, "")
        business_need_entry["Benefit"]["ReasonToBelieve"] = row.get(reason_col, "")
        business_need_entry["Benefit"]["TechnicalProof"] = row.get(proof_col, "")
        business_need_entry["Benefit"]["LegalNotice"] = row.get(legal_notice_col, "")

        # Append the business need entry to the French market model
        fr_market_model["BusinessNeeds"].append(business_need_entry)

    # Update or add the French market model in the segmentation JSON without removing other locales
    found_fr_market = False
    for market_model in segmentation_json.get("MarketModels", []):
        if market_model["LocaleId"] == "fr-FR":
            market_model.update(fr_market_model)
            found_fr_market = True
            break

    if not found_fr_market:
        segmentation_json.setdefault("MarketModels", []).append(fr_market_model)

    # Convert updated JSON to string and prepare for Excel output with "ProductSegmentationJson" as the column name
    segmentation_json_str = json.dumps(segmentation_json, ensure_ascii=False)
    excel_data.append({
        "ProductSegmentationJson": segmentation_json_str,
        "sys_id": product_entity_sys_id,
        "ProductGlobalId": product_global_id
    })

# Write updated data to Excel
df_output = pd.DataFrame(excel_data)
output_excel_path = "C:\\Users\\E106467\\Documents\\Json Mathilde\\Segmentation_Output_05.xlsx"
df_output.to_excel(output_excel_path, index=False)

print(f"The Excel file with JSON segmentation data has been successfully created and saved as {output_excel_path}")


The Excel file with JSON segmentation data has been successfully created and saved as C:\Users\E106467\Documents\Json Mathilde\Segmentation_Output_05.xlsx


In [3]:
import pandas as pd
import json

# Specify the LocaleId and LocaleName for the target country
locale_id = "bg-BG"  # Replace with the desired LocaleId (e.g., "bg-BG" for Bulgarian)
locale_name = "Bulgarian (Bulgaria)"  # Replace with the corresponding LocaleName

# Load the TLD data with multi-level headers
tld_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\filtered_Tld_B2B_modified (1).xlsx", header=[0, 1])
tld_data.columns = [' '.join(col).strip() for col in tld_data.columns.values]

# Load the CVL file without dropping any rows
cvl_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\CVL_B2B Benefit and Business Key.xlsx")

# Create mappings with NaN values handled gracefully
business_need_mapping = cvl_data.set_index('BusinessNeed_Key')['Business Need'].to_dict()
benefit_id_mapping = cvl_data.set_index('Benefit in your country')['Benefits_Key'].to_dict()

# Initialize list to store data for Excel output
excel_data = []

# Process each row in the TLD data
for _, row in tld_data.iterrows():
    product_global_id = row.get("Unnamed: 1_level_0 ProductGlobalID", "").strip()
    product_entity_sys_id = str(row.get("Unnamed: 0_level_0 sys_id", "")).strip()
    
    if not product_global_id or not product_entity_sys_id:
        print(f"Skipping row due to missing ProductGlobalID or sys_id: {row}")
        continue

    # Retrieve existing JSON from "Segmentation json" if present, otherwise create a new one
    existing_json_str = row.get("Unnamed: 3_level_0 Segmentation json", "{}").strip()
    try:
        segmentation_json = json.loads(existing_json_str) if existing_json_str else {}
    except json.JSONDecodeError:
        print(f"Warning: Malformed JSON for ProductGlobalId {product_global_id}. Attempting to repair.")
        # Attempt minimal repair
        if not existing_json_str.startswith("{"):
            existing_json_str = "{" + existing_json_str
        if not existing_json_str.endswith("}"):
            existing_json_str += "}"
        try:
            segmentation_json = json.loads(existing_json_str)
        except json.JSONDecodeError:
            print(f"Failed to repair JSON for ProductGlobalId {product_global_id}. Using default structure.")
            segmentation_json = {
                "ProductGlobalId": product_global_id,
                "ProductEntitySysId": product_entity_sys_id,
                "MarketModels": []
            }

    # Define the market model for the specified locale
    target_market_model = {
        "LocaleId": locale_id,
        "LocaleName": locale_name,
        "BusinessNeeds": []
    }

    # Process each BusinessNeed
    for business_need_id, business_need_name in business_need_mapping.items():
        if pd.isna(business_need_id) or pd.isna(business_need_name):
            continue  # Skip if either key or name is missing

        business_need_entry = {
            "BusinessNeedId": str(int(business_need_id)),
            "Benefit": {
                "BenefitId": "",
                "DisplayOrder": "",
                "ReasonToBelieve": "",
                "TechnicalProof": "",
                "LegalNotice": "",
                "ImageUrl": ""
            }
        }

        # Retrieve the BenefitId and other attributes based on column names with suffixes
        benefit_name = row.get(f"{business_need_name} Benefit in your country", "")
        if benefit_name in benefit_id_mapping:
            benefit_id = benefit_id_mapping[benefit_name]
            business_need_entry["Benefit"]["BenefitId"] = benefit_id

        # Define the column names with suffixes to get specific attributes
        display_order_col = f"{business_need_name} Sort order in your country (from 1 to 9)"
        reason_col = f"{business_need_name} Reason to believe in your language"
        proof_col = f"{business_need_name} Technical proof in your language"
        legal_notice_col = f"{business_need_name} Legal notice in your language"
        
        # Retrieve each attribute for the business need
        business_need_entry["Benefit"]["DisplayOrder"] = row.get(display_order_col, "")
        business_need_entry["Benefit"]["ReasonToBelieve"] = row.get(reason_col, "")
        business_need_entry["Benefit"]["TechnicalProof"] = row.get(proof_col, "")
        business_need_entry["Benefit"]["LegalNotice"] = row.get(legal_notice_col, "")

        # Append the business need entry to the target market model
        target_market_model["BusinessNeeds"].append(business_need_entry)

    # Update or add the specified market model in the segmentation JSON without removing other locales
    found_market_model = False
    for market_model in segmentation_json.get("MarketModels", []):
        if market_model["LocaleId"] == locale_id:
            market_model.update(target_market_model)
            found_market_model = True
            break

    if not found_market_model:
        segmentation_json.setdefault("MarketModels", []).append(target_market_model)

    # Convert updated JSON to string and prepare for Excel output with "ProductSegmentationJson" as the column name
    segmentation_json_str = json.dumps(segmentation_json, ensure_ascii=False)
    excel_data.append({
        "ProductSegmentationJson": segmentation_json_str,
        "sys_id": product_entity_sys_id,
        "ProductGlobalId": product_global_id
    })

# Write updated data to Excel
df_output = pd.DataFrame(excel_data)
output_excel_path = "C:\\Users\\E106467\\Documents\\Json Mathilde\\Segmentation_Output_006.xlsx"
df_output.to_excel(output_excel_path, index=False)

print(f"The Excel file with JSON segmentation data has been successfully created and saved as {output_excel_path}")


The Excel file with JSON segmentation data has been successfully created and saved as C:\Users\E106467\Documents\Json Mathilde\Segmentation_Output_006.xlsx


In [ ]:
# for multiple products with benefit key added
import pandas as pd
import json

# Load the data with MultiIndex headers
# tld_data est chargé avec deux en-têtes pour capturer les noms de colonne dans deux lignes (niveau 0 et niveau 1).
tld_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\filtered_Tld_B2B_1.xlsx", header=[0, 1])

# Flatten the MultiIndex headers into single-level by joining header levels with a space
# tld_data.columns refers to the column headers of the DataFrame tld_data.
#.values extracts these column names as an array of tuples, allowing them to be accessed directly.
#Using ' '.join(col) means joining the two header levels with a space (' '), so a tuple like ('Product ID', 'Global') becomes 'Product ID Global'.
# La méthode strip() supprime les espaces en début et en fin de chaîne dans le résultat final. Cela est utile si des espaces supplémentaires ont été ajoutés lors de la combinaison (join) des éléments.
tld_data.columns = [' '.join(col).strip() for col in tld_data.columns.values]

# Print flattened columns for verification
print("Flattened columns with both headers:", tld_data.columns)

# Load the CVL file and create mappings for Business Needs and Benefits
cvl_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\CVL_B2B Benefit and Business Key.xlsx")
#  La fonction dropna() de Pandas supprime les lignes avec des valeurs manquantes (NaN) dans un DataFrame.
cvl_data_cleaned = cvl_data.dropna(subset=['BusinessNeed_Key', 'Benefits_Key'])
# Create mappings from cleaned CVL data
business_need_mapping = cvl_data_cleaned.set_index('BusinessNeed_Key')['Business Need'].to_dict()  # Maps BusinessNeed_Key to Business Need
benefit_id_mapping = cvl_data_cleaned.set_index('Benefit in your country')['Benefits_Key'].to_dict()  # Maps Benefit in your country to Benefits_Key

# Initialize a list to store each product's data
products_data = []

# Process each product row in the TLD data
# Cette boucle for parcourt chaque ligne (ou produit) du DataFrame tld_data.
for _, row in tld_data.iterrows():
    # Retrieve ProductGlobalID and sys_id using flattened names
    product_global_id = row.get("Unnamed: 1_level_0 ProductGlobalID", "").strip()
    product_entity_sys_id = str(row.get("Unnamed: 0_level_0 sys_id", "")).strip()
    
    # Ensure both identifiers are present
    if not product_global_id or not product_entity_sys_id:
        print(f"Skipping row due to missing ProductGlobalID or sys_id: {row}")
        continue

    # Initialize product structure
    product_data = {
        "ProductGlobalId": product_global_id,
        "ProductEntitySysId": product_entity_sys_id,
        "MarketModels": [
            {
                "LocaleId": "fr-FR",
                "LocaleName": "French (France)",
                "BusinessNeeds": []
            }
        ]
    }

    # Process each BusinessNeed
    # business_need_mapping.items() : La méthode .items() retourne les paires clé-valeur sous forme de tuples du mapping  (clé, valeur).
  
    for business_need_id, business_need_name in business_need_mapping.items():
          # Crée un dictionnaire business_need_entry qui représente la structure de chaque besoin métier (BusinessNeed)
        # à ajouter dans le fichier JSON parce que le json prends un entier .
        business_need_entry = {
            "BusinessNeedId": str(int(business_need_id)),
            "Benefit": {
                "BenefitId": "",
                "DisplayOrder": "",
                "ReasonToBelieve": "",
                "TechnicalProof": "",
                "LegalNotice": "",
                "ImageUrl": ""
            }
        }

        # Retrieve the BenefitId and other attributes based on column names with suffixes
        benefit_name = row.get(f"{business_need_name} Benefit in your country", "")
        if benefit_name in benefit_id_mapping:
            benefit_id = benefit_id_mapping[benefit_name]
            business_need_entry["Benefit"]["BenefitId"] = benefit_id

        # Define the column names with suffixes to get specific attributes
        display_order_col = f"{business_need_name} Sort order in your country (from 1 to 9)"
        reason_col = f"{business_need_name} Reason to believe in your language"
        proof_col = f"{business_need_name} Technical proof in your language"
        legal_notice_col = f"{business_need_name} Legal notice in your language"
        
        # Retrieve each attribute for the business need
        # La méthode row.get() va chercher dans la ligne (row) une colonne 
        # qui correspond au nom de colonne exact, construit dynamiquement comme ceci : {business_need_name} Sort order in your country (from 1 to 9).
        business_need_entry["Benefit"]["DisplayOrder"] = row.get(display_order_col, "")
        business_need_entry["Benefit"]["ReasonToBelieve"] = row.get(reason_col, "")
        business_need_entry["Benefit"]["TechnicalProof"] = row.get(proof_col, "")
        business_need_entry["Benefit"]["LegalNotice"] = row.get(legal_notice_col, "")

        # Append the business need to the product's MarketModels
        # adding a detailed business need entry (business_need_entry) to the "BusinessNeeds" 
        # list within the first market model ("fr-FR") of the product data, ensuring that all 
        # relevant information for that business need is included in the product’s JSON structure.
        product_data["MarketModels"][0]["BusinessNeeds"].append(business_need_entry)

    # Append the complete product data to the products list
    products_data.append(product_data)

# Save the updated JSON with multiple products
 # sauvegarde les données des produits en format JSON lisible  dans le fichier Updated_B2B_multiple_products_1.json.
output_file = "C:\\Users\\E106467\\Documents\\Json Mathilde\\Updated_B2B_multiple_products_1.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(products_data, f, ensure_ascii=False, indent=4)

print(f"The JSON file with multiple products has been successfully created and saved as {output_file}")

In [1]:
# if we want to change the local id and name for different countries 
import pandas as pd
import json

# Définir les valeurs de localisation dynamiques
locale_id = "ar-SA"
locale_name = "Arabic (Saudi Arabia)"

# Charger les données avec les en-têtes multi-niveaux
tld_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\filtered_Tld_B2B_1.xlsx", header=[0, 1])
tld_data.columns = [' '.join(col).strip() for col in tld_data.columns.values]

# Charger le fichier CVL et créer les mappings pour les Business Needs et les Benefits
cvl_data = pd.read_excel("C:\\Users\\E106467\\Documents\\Json Mathilde\\CVL_B2B Benefit and Business Key.xlsx")
cvl_data_cleaned = cvl_data.dropna(subset=['BusinessNeed_Key', 'Benefits_Key'])
business_need_mapping = cvl_data_cleaned.set_index('BusinessNeed_Key')['Business Need'].to_dict()
benefit_id_mapping = cvl_data_cleaned.set_index('Benefit in your country')['Benefits_Key'].to_dict()

# Initialiser une liste pour stocker les données de chaque produit
products_data = []

# Traiter chaque ligne de produit dans les données TLD
for _, row in tld_data.iterrows():
    product_global_id = row.get("Unnamed: 1_level_0 ProductGlobalID", "").strip()
    product_entity_sys_id = str(row.get("Unnamed: 0_level_0 sys_id", "")).strip()

    if not product_global_id or not product_entity_sys_id:
        print(f"Skipping row due to missing ProductGlobalID or sys_id: {row}")
        continue

    # Créer la structure de données du produit avec les valeurs de localisation
    product_data = {
        "ProductGlobalId": product_global_id,
        "ProductEntitySysId": product_entity_sys_id,
        "MarketModels": [
            {
                "LocaleId": locale_id,
                "LocaleName": locale_name,
                "BusinessNeeds": []
            }
        ]
    }

    # Traiter chaque BusinessNeed
    for business_need_id, business_need_name in business_need_mapping.items():
        business_need_entry = {
            "BusinessNeedId": str(int(business_need_id)),
            "Benefit": {
                "BenefitId": "",
                "DisplayOrder": "",
                "ReasonToBelieve": "",
                "TechnicalProof": "",
                "LegalNotice": "",
                "ImageUrl": ""
            }
        }

        # Récupérer l'ID de bénéfice et les autres attributs
        benefit_name = row.get(f"{business_need_name} Benefit in your country", "")
        if benefit_name in benefit_id_mapping:
            benefit_id = benefit_id_mapping[benefit_name]
            business_need_entry["Benefit"]["BenefitId"] = benefit_id

        display_order_col = f"{business_need_name} Sort order in your country (from 1 to 9)"
        reason_col = f"{business_need_name} Reason to believe in your language"
        proof_col = f"{business_need_name} Technical proof in your language"
        legal_notice_col = f"{business_need_name} Legal notice in your language"

        business_need_entry["Benefit"]["DisplayOrder"] = row.get(display_order_col, "")
        business_need_entry["Benefit"]["ReasonToBelieve"] = row.get(reason_col, "")
        business_need_entry["Benefit"]["TechnicalProof"] = row.get(proof_col, "")
        business_need_entry["Benefit"]["LegalNotice"] = row.get(legal_notice_col, "")

        # Ajouter le besoin à la liste BusinessNeeds
        product_data["MarketModels"][0]["BusinessNeeds"].append(business_need_entry)

    # Ajouter le produit terminé à la liste principale des produits
    products_data.append(product_data)

# Enregistrer le fichier JSON avec les produits multiples
output_file = "C:\\Users\\E106467\\Documents\\Json Mathilde\\Updated_B2B_multiple_products_ar-SA.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(products_data, f, ensure_ascii=False, indent=4)

print(f"Le fichier JSON avec plusieurs produits pour {locale_name} a été créé et sauvegardé avec succès sous {output_file}")


Le fichier JSON avec plusieurs produits pour Arabic (Saudi Arabia) a été créé et sauvegardé avec succès sous C:\Users\E106467\Documents\Json Mathilde\Updated_B2B_multiple_products_ar-SA.json
